# Types

In Julia everything is an object and every object has a type. Most importantly, an object's type determines which functions accept that object as in input.

In [4]:
!3 #wrong type

MethodError: MethodError: no method matching !(::Int64)

Closest candidates are:
  !(!Matched::Missing)
   @ Base missing.jl:101
  !(!Matched::Bool)
   @ Base bool.jl:35
  !(!Matched::ComposedFunction{typeof(!)})
   @ Base operators.jl:1099
  ...


In Julia, users have a lot of control over structuring objects and determining which functions apply to which objects. This means we can create types and define function on them! Before learning that let's first take a closer look at types already existing in Julia.

### Numbers

We will work the most with integers and floating point numbers (these express decimal fractions).

In [5]:
[typeof(3), typeof(3.0)]

2-element Vector{DataType}:
 Int64
 Float64

64 means that information is stored in 64 bits. Means 2^64 different numbers of each kind. This is
 - a lot,
 - but **finite**!

No need to know technicalities, but useful to keep in mind certain consequences of the finiteness of computers.

1. There exist largest and smallest numbers! (can find maximums by running `typemax(Int64)` and `floatmax(Float64)`) Going over bounds will result in very strange behavior:

In [6]:
10^12321354563

0

In [7]:
10.1^12321354563

Inf

Btw `Inf` is a Float64 'number' which is defined to be larger than any number:

In [8]:
Inf>10^125

true

2. Floating point numbers are not dense (in the topological sense): For any floating point number $x$ there is an $eps(x)$ such that no floating point number between $x$ and $x + eps(x)$.

> #### Important:
> floating point numbers are more dense the closer they are to 0, so doing operations with large numbers is usually less precise.

In [9]:
eps(0.0)

5.0e-324

means that the next number after 0 is $5*e^{-324}$. One can live with this. For numbers with big absolute values the increments are larger:

In [10]:
eps(100000.0)

1.4551915228366852e-11

The consequence is that floating point operations are usually not completely precise, but involve some rounding errors:

In [11]:
1.0-0.8

0.19999999999999996

In [12]:
println(0.2-(1.0-0.8))

eps(0.2)

5.551115123125783e-17


2.7755575615628914e-17

Since this was a rather simple operation, the size of numerical error is the same magnitude as suggested by `eps`. In a more complex example, numerical errors can amplify each other.

### Strings and characters

Strings are finite sequences of characters. We define them using `""`.

In [13]:
st = "This is a string."

"This is a string."

In [14]:
println(typeof(st))
println(st[2]) # second character of st is an 'h'
println(typeof(st[2]))

String
h
Char


Anything between `""` signs is a string and hence cannot be used for other things. `"14"` is still a string, not a number!  

In [15]:
sqrt("14") # won't work

MethodError: MethodError: no method matching sqrt(::String)

Closest candidates are:
  sqrt(!Matched::BigInt)
   @ Base mpfr.jl:644
  sqrt(!Matched::Float16)
   @ Base math.jl:1558
  sqrt(!Matched::BigFloat)
   @ Base mpfr.jl:636
  ...


This particular string can however be converted into a number

In [16]:
sqrt(parse(Int64,"14")) # this works

3.7416573867739413

turning numbers into strings is more intuitive:

In [17]:
string(1.4)

"1.4"

Strings are iterables, that's why `st[2]` returns something sensible. We can therefore iterate over them!

In [18]:
for s in st
    println(uppercase(s))
end

T
H
I
S
 
I
S
 
A
 
S
T
R
I
N
G
.


we can also take a subset of them like of vectors:

In [19]:
st[1:4]

"This"

There are plenty of functions operating on strings. We cover two important features:

##### Concatenation of strings

One can attach strings together by the `*` operator

In [20]:
"Witho"*"ut"*" "*"interru"*"ption"

"Without interruption"

##### String interpolation

Sometimes we want to combine strings with the result of some (potentially non-string) expression. We could usually convert our result to string and then use `*`, but the so-called sting interpolation is more convenient:

In [21]:
function enthusiastic_summing(x,y)
    println("Let's add up $x and $y ! Wow, we got $(x+y)!!!")
    return x+y
end

enthusiastic_summing(2,5)

Let's add up 2 and 5 ! Wow, we got 7!!!


7

## End of Tuesday lecture in this file

#### Exercise

Write a function that takes a positive integer $n$, and for each integer $i$ from $1$ to $n$ it prints the following sentence: The sum of the first $i$ positive integers is ... .

In [1]:
# work here
function printsum(n)
    s = 0
    for i in 1:n
        s = s + i
        println("The sum of the first $i positive integers is $s")
    end
end

printsum(8)

The sum of the first 1 positive integers is 1
The sum of the first 2 positive integers is 3
The sum of the first 3 positive integers is 6
The sum of the first 4 positive integers is 10
The sum of the first 5 positive integers is 15
The sum of the first 6 positive integers is 21
The sum of the first 7 positive integers is 28
The sum of the first 8 positive integers is 36


In [ ]:
# next class start with this

using Pkg
Pkg.add("TypeTree")
Pkg.add("Roots")
Pkg.add("Optim")
Pkg.add("DataFrames")
Pkg.add("Interpolations")
Pkg.add("CSV")

### Types and Methods   

To emphasize how general types we can create, consider the following example. First we create an abstract type for animals and two concrete subtypes of it: one for dogs and one for cats.

In [3]:
abstract type AbstractAnimal end

struct Dog<:AbstractAnimal    # Dog is a subtype of AbstractAnimal
    name::String              # name must be a string
    age::Integer
    shoes::Integer
end

struct Cat<:AbstractAnimal 
    name::String
    age::Integer
    victim::String
end

Let's create some example animals: 

In [4]:
a = Dog("Ginger",11,3)

b = Cat("Molly", 4, "tiny birds")

c = Cat("Kitty", 2, "")

Cat("Kitty", 2, "")

In [8]:
b.name

"Molly"

Now we create some functions:

In [6]:
function name(a::AbstractAnimal) # this method applies to inputs, which are belong to some subtype of AbstractAnimal 
    return a.name
end

function name(c::Cat) # this method applies to inputs, which are cats!
    if length(c.victim)>0
        return c.name*" the Killer"
    else
        return c.name*" the Innocent"
    end
end

name (generic function with 2 methods)

When applying `name(animal)` Julia looks for the most specific function method that applies to the particular species of animal.
 - For dogs, only the generic function for any animal works, so that is applied.
 - For cats, the more specific function for cats will be called.

In [7]:
println(name(a))
println(name(b))
println(name(c))

Ginger
Molly the Killer
Kitty the Innocent


In [ ]:
function intro(a::AbstractAnimal)
    return "My name is $(name(a)), I am $(a.age) years old and I enjoy $(main_activity(a))."
end

function main_activity(d::Dog)
    return "chewing at least $(d.shoes) shoes per day"
end

function main_activity(c::Cat)
    if length(c.victim) > 0
        return "massacring "*c.victim
    else
        return "sleeping"
    end
end

`intro` can be called on any type of animal. But inside, those methods of `name` and `main_activity` are called, which suit best the tyep of the original input of `intro`.

In [ ]:
println(intro(a))
println(intro(b))
println(intro(c))

### Types of Types

- Abstract types
- Concrete types

The main difference is that one cannot create an object that is of an abstract type. In other words, every object belongs to some concrete type. Abstract types are only used to organize concrete types in a tree, to make it simpler to write general functions (consider our function `intro` above as an example, acting on abstract type `AbstractAnimal`).

In [9]:
typeof(3.1)

Float64

In [10]:
typeof(3)

Int64

Both Float64 and Int64 are subtype of an abstract type Number

In [11]:
Float64<:Number && Int64<:Number

true

So how many types of numbers exist? A lot! Let's take a look!

In [ ]:
#using Pkg
#Pkg.add("TypeTree")


In [12]:
using TypeTree
println(join(tt(Number), ""))

Number
 ├─ Base.MultiplicativeInverses.MultiplicativeInverse
 │   ├─ Base.MultiplicativeInverses.SignedMultiplicativeInverse
 │   └─ Base.MultiplicativeInverses.UnsignedMultiplicativeInverse
 ├─ Complex
 └─ Real
     ├─ AbstractFloat
     │   ├─ BigFloat
     │   ├─ Float16
     │   ├─ Float32
     │   └─ Float64
     ├─ AbstractIrrational
     │   └─ Irrational
     ├─ Integer
     │   ├─ Bool
     │   ├─ Signed
     │   │   ├─ BigInt
     │   │   ├─ Int128
     │   │   ├─ Int16
     │   │   ├─ Int32
     │   │   ├─ Int64
     │   │   └─ Int8
     │   └─ Unsigned
     │       ├─ UInt128
     │       ├─ UInt16
     │       ├─ UInt32
     │       ├─ UInt64
     │       └─ UInt8
     └─ Rational



In [13]:
supertype(Number)

Any

In [14]:
println(join(tt(AbstractAnimal), ""))
println("supertype of AbstractAnimal is $(supertype(AbstractAnimal))")

AbstractAnimal
 ├─ Cat
 └─ Dog

supertype of AbstractAnimal is Any


### Types of Concrete types

- Primitive types
- Composite types

Primitive types pertain in which format exactly data is stored in the memory, so common users (like us) never need to define them, we just use them. Examples include `Char` and all concrete subtypes of `Number`, such as `Bool`, `Int64` or `Float64`.

Composite types consist of elements (called 'fields'), which are either of some primitive type or belong to some other composite type. `Cat` and `Dog` are composite concrete types, and so are Arrays. Before getting more acquainted with some built-in composite types in Julia, let's note that some of them are immutable, while others are mutable. Fields of immutable objects cannot be changed after the object is created. Composite types created by the `struct` keyword are immutable.

In [15]:
println("name of 'a' is currently $(a.name)")
a.name = "Fluffy" # this fails

name of 'a' is currently Ginger


ErrorException: setfield!: immutable struct of type Dog cannot be changed

### Arrays (again)

Arrays are mutable structures. This means that their content can be modified after creation.

In [16]:
v = [1, 2]

2-element Vector{Int64}:
 1
 2

In [17]:
v[1] = 3
v

2-element Vector{Int64}:
 3
 2

But you can cannot change the type of an existing array. `v` is a vector of Int64 elements, so this fails:

In [18]:
v[2] = "2" 

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Int64

Closest candidates are:
  convert(::Type{T}, !Matched::T) where T<:Number
   @ Base number.jl:6
  convert(::Type{T}, !Matched::T) where T
   @ Base Base.jl:84
  convert(::Type{T}, !Matched::Number) where T<:Number
   @ Base number.jl:7
  ...


one could initiate a mixed array though: (it is rarely a good idea nevertheless)

In [22]:
vec = [1, "2", 3]

3-element Vector{Any}:
 1
  "2"
 3

In [20]:
typeof.([1, "2"])

2-element Vector{DataType}:
 Int64
 String

In [25]:
convert.(Int64,vec[[1,3]])

2-element Vector{Int64}:
 1
 3

### Adventures with assignment (`=`)

Time to get familiar with a more confusing aspect of Julia. This is probably intuitive:

In [26]:
a = 1
b = a # b gets the same value as a
println("a is $a, b is $b")
b = 2
println("now a is $a, b is $b")

a is 1, b is 1
now a is 1, b is 2


Same with vectors shows a different behavior. Changing `b` affects `a`.

In [27]:
a = [1, 1]
b = a
println("a is $a, b is $b")
b[2] = 3
println("now a is $a, b is $b")

a is [1, 1], b is [1, 1]
now a is [1, 3], b is [1, 3]


- What's going on? When seeing `[1, 1]`, Julia puts two `1`s somewhere in the memory and notes their address. Then creates a pointer the the location where this data lives and save this pointer under the name `a`.  `b = a` copies the pointer to name `b` as well, but it still points the **same** place on the memory. So when `b[2] = 3` updates the data on the location where both `a` and `b` points, they both change.

- This looks annoying. Why does Julia do this? Julia wants to be fast. For that it is crucial not to copy arrays unless necessary. So not copying is the default behavior.

- Why behavior is different when `a` is a number versus an array? Because numbers are stored differently, and copying them is not a performance issue.

**TLDR**: `a` is not really a vector, it is only a pointer to a vector. When running `b=a`, the pointer is copied, not the vector.

Sometimes you do want to copy the contents of an array. Then you can do this:

In [28]:
a = [1, 1]
b = copy(a) # like this 'b' will point to a copy of the vector that 'a' points to
println("a is $a, b is $b")
b[2] = 3
println("now a is $a, b is $b")

a is [1, 1], b is [1, 1]
now a is [1, 1], b is [1, 3]


### Tuples

In Julia ordered collections of variables in round brackets are called tuples.

In [29]:
a = (1,2)
typeof(a)

Tuple{Int64, Int64}

We just need to know a couple of things about them. First, since the type of its elements are kept track of separately, it isn't a crime to combine different types in them:

In [30]:
b = (1,"2")
typeof(b)

Tuple{Int64, String}

we can access the elements of tuples like that of arrays:

In [31]:
b[1]

1

Tuples are immutable: Once created, its impossible to amend them.

In [32]:
b[1] = 3

MethodError: MethodError: no method matching setindex!(::Tuple{Int64, String}, ::Int64, ::Int64)

Tuples often provide the best way to write a function returning several variables.

In [34]:
function twooutput(n)
    if n<0
        m = "negative"
    else
        m = "non-negative"
    end
    return (n^2,m) # brackets here are optional, simply listing the two things would also return a tuple
end

op = twooutput(3)
println(op[1])
println(op[2])

9
non-negative


so the output is a tuple:

In [35]:
op

(9, "non-negative")

Even more conveniently, we can directly access the outputs like this:

In [36]:
(square, sign) = twooutput(3)

(9, "non-negative")

In [37]:
square

9

In [38]:
sign

"non-negative"